In [1]:
from data_loader import *
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as func
import torch.nn.init as torch_init
import torch.optim as optim
import shutil
import pickle

class LSTM(nn.Module):
    """ A basic LSTM model. 
    """
    
    def __init__(self, in_dim, out_dim, hid_dim, batch_size, no_layers =1):
        super(LSTM, self).__init__()
        #specify the input dimensions
        self.in_dim = in_dim
        #specify the output dimensions
        self.out_dim = out_dim
        #specify hidden layer dimensions
        self.hid_dim = hid_dim
        #specify the number of layers
        self.no_layers = no_layers  
        #self.batch_size=batch_size
        
        #initialise the LSTM
        self.model = nn.LSTM(self.in_dim, self.hid_dim, self.no_layers)
        self.outputs = nn.Linear(self.hid_dim, out_dim)

    def forward(self, batch,hidden=None):
        """Pass the batch of images through each layer of the network, applying 
        """
        
        lstm_out, hidden = self.model(batch, hidden)
        y_pred = self.outputs(lstm_out.view(lstm_out.shape[0]*lstm_out.shape[1],lstm_out.shape[-1]))
        #pdb.set_trace()
        #The input is expected to contain raw, unnormalized scores for each class according to documentation
        #tag_scores = func.softmax(y_pred,dim=2)
        #return tag_scores,hidden
        return y_pred,hidden
    
def validate(val_loader, model, criterion):
    losses = AverageMeter()
    acc = AverageMeter()

    # switch to evaluate mode
    model.eval()  
    hidden=None
    for minibatch_count, (images, labels) in enumerate(val_loader, 0):
        images=images.permute(1,0,2)
        # Put the minibatch data in CUDA Tensors and run on the GPU if supported
        images, labels = images.to(computing_device), labels.to(computing_device)
        images=images.float()
       
        outputs,hidden = model(images,hidden)
        hidden[0].detach_() 
        hidden[1].detach_()
        
        labels=labels.view(-1)
        loss = criterion(outputs, labels)
        
        softmaxOutput=func.softmax(outputs,dim=1)
        modelDecision=torch.argmax(softmaxOutput,dim=1)
        accuracy = calculate_acc(modelDecision,labels)
        losses.update(loss.item(), labels.shape[0])
        acc.update(accuracy, labels.shape[0])

    return acc,losses
class AverageMeter(object):

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def calculate_acc(modelDecision,labels):
    acc=torch.sum(labels==modelDecision).to(dtype=torch.float)/labels.shape[0]
    return acc

def save_checkpoint(state, is_best, filename='checkpoint.pth.tar'):
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, 'model_best.pth.tar')
        
        
# Setup: initialize the hyperparameters/variables
num_epochs = 100           # Number of full passes through the dataset
batch_size = 1          # Number of samples in each minibatch
learning_rate = 0.001  
#use_cuda=0
use_cuda=torch.cuda.is_available()

# Setup GPU optimization if CUDA is supported
if use_cuda:
    computing_device = torch.device("cuda")
    extras = {"num_workers": 1, "pin_memory": True}
    print("CUDA is supported")
else: # Otherwise, train on the CPU
    computing_device = torch.device("cpu")
    extras = False
    print("CUDA NOT supported")
    

CUDA is supported


In [2]:
def toAbc1(numLabel,dictionary):
    #take in onehot encoded data and transform it to abc

    dic_val_list=list(dictionary.values())
    dic_key_list=list(dictionary.keys()) 
    abc=dic_key_list[dic_val_list.index(numLabel)]
    #can be commented    
    return abc

model = LSTM(in_dim=94, out_dim=94,hid_dim=100,batch_size=16,no_layers=1)
model = model.to(computing_device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
checkpoint = torch.load('model_best.pth.tar')
begin_epoch = checkpoint['epoch']
best_prec1 = checkpoint['best_loss']
model.load_state_dict(checkpoint['state_dict'])
optimizer.load_state_dict(checkpoint['optimizer'])
dictionary=pickle.load(open("dictionary_new.pkl","rb"))
# from bidict import bidict 
# reverse_dict = bidict(dictionary)
# reverse_dict.inverse

def sample(net, size, prime, dictionary):
        
    if(use_cuda):
        computing_device = torch.device("cuda")
    else:
        computing_device = torch.device("cpu")
    net = net.to(computing_device)
    net.eval() 
    
    h = None
    chars=list(prime)
    lis=[]
    for ch in chars:
        chNum=dictionary[ch]
        lis.append(chNum)
    
    lis=np.array(lis)
    softmaxLayer=torch.nn.Softmax()
    #pdb.set_trace()
    #a=torch.tensor(np.squeeze(np.eye(94)[lis.reshape(-1)]).reshape([len(prime), 1, 94])).to(computing_device).float()
    #pdb.set_trace()
    inputPrim=torch.tensor(fromNumLabelToOneHot(lis,dictionary).reshape([len(prime), 1, 94])).to(computing_device).float()
    with torch.no_grad():
        char, h = net(inputPrim, None )        
        
        #pdb.set_trace()
         
        softmaxOutput=func.softmax(char[-1,:])
        p_p=softmaxOutput.cpu()
        p_p = np.array(p_p) 
        Val=np.random.choice(94, 1,p=p_p.reshape(94,))
        chars.append(toAbc1(Val,dictionary))
        newInput=fromNumLabelToOneHot(Val,dictionary)
        for j in range(size):     
            inputPrimNew=torch.tensor(np.zeros((100,1,94),dtype=int))
            #pdb.set_trace()
            inputPrimNew[:-1,:]=inputPrim[1:,:]
            inputPrimNew[-1,:]=torch.tensor(newInput)
            inputPrim=inputPrimNew.to(computing_device).float()
            char,h=net(inputPrim.view(len(prime),1,94),h)
            softmaxOutput=func.softmax(char[-1,:])
            p_p=softmaxOutput.cpu()
            p_p = np.array(p_p) 
            Val=np.random.choice(94, 1,p=p_p.reshape(94,))
            chars.append(toAbc1(Val,dictionary))
            newInput=fromNumLabelToOneHot(Val,dictionary)
            #pdb.set_trace()
    return chars
        
result=(sample(model, 2000, '<start>\nX:19\nT:Dusty Miller, The\nR:hop jig\nD:Tommy Keane: The Piper\'s Apron\nZ:id:hn-slipjig-19\nM:9/8', dictionary))

with open('generated.txt', 'w') as f:
    for item in result:
        f.write("%s" % item)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:50: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:63: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [3]:
result

['<',
 's',
 't',
 'a',
 'r',
 't',
 '>',
 '\n',
 'X',
 ':',
 '1',
 '9',
 '\n',
 'T',
 ':',
 'D',
 'u',
 's',
 't',
 'y',
 ' ',
 'M',
 'i',
 'l',
 'l',
 'e',
 'r',
 ',',
 ' ',
 'T',
 'h',
 'e',
 '\n',
 'R',
 ':',
 'h',
 'o',
 'p',
 ' ',
 'j',
 'i',
 'g',
 '\n',
 'D',
 ':',
 'T',
 'o',
 'm',
 'm',
 'y',
 ' ',
 'K',
 'e',
 'a',
 'n',
 'e',
 ':',
 ' ',
 'T',
 'h',
 'e',
 ' ',
 'P',
 'i',
 'p',
 'e',
 'r',
 "'",
 's',
 ' ',
 'A',
 'p',
 'r',
 'o',
 'n',
 '\n',
 'Z',
 ':',
 'i',
 'd',
 ':',
 'h',
 'n',
 '-',
 's',
 'l',
 'i',
 'p',
 'j',
 'i',
 'g',
 '-',
 '1',
 '9',
 '\n',
 'M',
 ':',
 '9',
 '/',
 '8',
 '\n',
 'L',
 ':',
 '1',
 '/',
 '8',
 '\n',
 'K',
 ':',
 'V',
 '\n',
 'f',
 'f',
 ' ',
 'f',
 '>',
 'f',
 '|',
 'e',
 '>',
 'd',
 ' ',
 'c',
 'A',
 '|',
 'd',
 'B',
 ' ',
 'A',
 'd',
 '|',
 'e',
 'f',
 ' ',
 'f',
 'g',
 '|',
 'a',
 'd',
 ' ',
 'f',
 'a',
 '|',
 'f',
 '/',
 'a',
 '/',
 'g',
 '/',
 'f',
 '/',
 ' ',
 'e',
 'f',
 'e',
 '|',
 'e',
 'd',
 ' ',
 'B',
 'A',
 '|',
 'd',
 '2',
 ' ',
 